<a href="https://colab.research.google.com/github/juhumkwon/YOLO/blob/main/1_1_YOLO%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5%EA%B3%BC%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
file_path = '/content/drive/My Drive'
print(os.listdir(file_path))


In [ ]:
!pip install torch torchvision numpy pandas opencv-python
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5
!pip install -r requirements.txt


In [ ]:
# YOLOv5 전이 학습

import os
from pathlib import Path

# YOLOv5 라이브러리 가져오기
from yolov5 import train

# 커스텀 데이터셋 경로
DATASET_YAML = "path/to/your/dataset.yaml"  # YAML 파일 경로 (예: 'data/coco128.yaml')
PRETRAINED_MODEL = "yolov5s.pt"  # 사전 학습된 모델 (yolov5s, yolov5m 등)

# 저장 경로 설정
SAVE_DIR = "runs/train_custom"  # 학습 결과 저장 경로

# 학습 파라미터 설정
EPOCHS = 50  # 학습 반복 횟수
BATCH_SIZE = 16  # 배치 크기
IMG_SIZE = 640  # 입력 이미지 크기

def main():
    # YOLOv5 훈련 실행
    train.run(
        data=DATASET_YAML,
        weights=PRETRAINED_MODEL,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
        project=SAVE_DIR,
        name="exp",
    )

if __name__ == "__main__":
    main()


In [ ]:
# 데이터셋 준비
# 데이터셋은 YOLO 포맷(이미지와 라벨 파일)이어야 하며, dataset.yaml 파일로 정의해야 합니다.

train: ../data/train/images  # 학습 이미지 경로
val: ../data/val/images      # 검증 이미지 경로

# 클래스 이름 (예: 2개 클래스인 경우)
names:
  0: cat
  1: dog


In [ ]:
# 추론 코드
# YOLOv5로 학습된 모델을 이용하여 추론하는 간단한 예제 프로그램을 아래에 제공합니다.
# 이 코드는 학습된 모델(best.pt)을 사용하여 이미지나 동영상에서 객체를 감지합니다.

import torch
from pathlib import Path
import cv2
import matplotlib.pyplot as plt

# 학습된 모델 경로
MODEL_PATH = "runs/train_custom/exp/weights/best.pt"  # 학습된 모델 파일 경로
IMAGE_PATH = "path/to/your/image.jpg"  # 추론할 이미지 경로
VIDEO_PATH = "path/to/your/video.mp4"  # 추론할 동영상 경로 (옵션)

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH)  # 사용자 정의 모델 로드

def infer_image(image_path):
    # 이미지 로드 및 추론
    results = model(image_path)

    # 추론 결과 시각화
    results.render()  # 이미지 위에 박스와 라벨 그리기
    for img in results.imgs:
        plt.imshow(img[..., ::-1])  # BGR -> RGB로 변환
        plt.axis("off")
        plt.show()

def infer_video(video_path, output_path="output.mp4"):
    # 동영상 로드
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 코덱
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 추론 수행
        results = model(frame)

        # 추론 결과 이미지 위에 그리기
        for box in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = map(int, box[:6])
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        out.write(frame)  # 결과 저장
        cv2.imshow("YOLOv5 Inference", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # 이미지 추론
    infer_image(IMAGE_PATH)

    # 동영상 추론 (옵션)
    # infer_video(VIDEO_PATH, output_path="output_inference.mp4")


In [ ]:
# 이미지 추론 예제 코드

import torch
import cv2
import matplotlib.pyplot as plt

# 학습된 모델 경로
MODEL_PATH = "runs/train_custom/exp/weights/best.pt"  # 학습된 모델 파일 경로
IMAGE_PATH = "path/to/your/image.jpg"  # 추론할 이미지 경로

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH)  # 사용자 정의 모델 로드

def infer_image(image_path):
    # 이미지 로드 및 추론
    results = model(image_path)

    # 추론 결과 시각화
    results.render()  # 이미지 위에 박스와 라벨 그리기

    # 결과 이미지 시각화
    for img in results.imgs:
        plt.imshow(img[..., ::-1])  # BGR -> RGB로 변환
        plt.axis("off")
        plt.show()

if __name__ == "__main__":
    # 이미지 추론
    infer_image(IMAGE_PATH)
